## Company classifier notebook

This is for experimenting with the company fit classifier.
Visualizations, etc can go here.

In [3]:
print("hello world")

hello world


### Setup and Imports

In [5]:
import sys
import os
# Add parent directory to Python path so we can import company_classifier
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from company_classifier.classifier import (
    BAD_FIT,
    GOOD_FIT,
    NEED_MORE_INFO,
    CompanyClassifier,
)

### Load and Prepare Data

In [7]:
def load_real_data(csv_path: str) -> tuple[pd.DataFrame, np.ndarray]:
    """Load and prepare real company data."""
    df = pd.read_csv(csv_path)

    # Map text categories to numeric values
    category_map = {"good": GOOD_FIT, "bad": BAD_FIT, "needs_more_info": NEED_MORE_INFO}

    # Extract features used by our classifier
    X = df[
        [
            "type",
            "total_comp",
            "base",
            "rsu",
            "bonus",
            "remote_policy",
            "eng_size",
            "total_size",
        ]
    ]

    # Convert fit categories to numeric values
    y = df["fit_category"].map(category_map).to_numpy()

    return X, y

# Load the data
X, y = load_real_data("../company_ratings.csv")

# Show dataset info
print(f"Dataset size: {len(X)} companies\n")
print("Class distribution:")
classes, counts = np.unique(y, return_counts=True)
for cls, count in zip(classes, counts):
    label = {
        GOOD_FIT: "Good fit",
        BAD_FIT: "Bad fit",
        NEED_MORE_INFO: "Need more info",
    }[cls]
    print(f"{label}: {count}")

Dataset size: 47 companies

Class distribution:
Bad fit: 9
Good fit: 22
Need more info: 16


### Train-Test Split and Initial Evaluation

In [8]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Split sizes:")
print(f"Training set: {len(X_train)} companies")
print(f"Test set: {len(X_test)} companies")

# Train and evaluate
print("\nTraining and evaluating classifier...")
classifier = CompanyClassifier()
classifier.fit(X_train, y_train)

# Test set performance
y_pred = classifier.predict(X_test)
print("\nTest Set Performance:")
print(
    classification_report(
        y_test, y_pred, target_names=["Bad fit", "Good fit", "Need more info"]
    )
)

Split sizes:
Training set: 37 companies
Test set: 10 companies

Training and evaluating classifier...

Test Set Performance:
                precision    recall  f1-score   support

       Bad fit       1.00      1.00      1.00         2
      Good fit       0.71      1.00      0.83         5
Need more info       1.00      0.33      0.50         3

      accuracy                           0.80        10
     macro avg       0.90      0.78      0.78        10
  weighted avg       0.86      0.80      0.77        10



/Users/paul/src/recruiter-messager/.direnv/python-3.13/lib/python3.13/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


### Cross-Validation Analysis

### Feature Importance Analysis
